### 버전 확인 및 패키지 설치

In [ ]:
!which python
!where python
!pip show pandas
!export PYTHONPATH=$PYTHONPATH:/opt/anaconda3/bin/python
import sys
sys.path.append('/opt/anaconda3/lib/python3.12/site-packages')



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
import random
import selenium
import pandas as pd

# 전체 출력 설정
pd.set_option('display.max_rows', None)  # 모든 행 출력
pd.set_option('display.max_columns', None)  # 모든 열 출력

### 셀레니움 드라이버 생성

In [ ]:

# 크롬 드라이버 경로
driver_path = r'/Users/apple/Desktop/coding_file/파이썬 크롤링/chromedriver-mac-x64/chromedriver'

# Service 객체 생성
service = Service(driver_path)

# WebDriver 초기화
driver = webdriver.Chrome(service=service)

# URL 열기
URL = "https://www.celine.com/ko-kr/celine-여성/핸드백/?nav=A003-VIEW-ALL"
driver.get(URL)

### 무한 스크롤
- 스크롤 이전, 이후 페이지 높이가 같으면 스크롤 멈춤

In [ ]:
def infinite_loop():
    #스크롤 내리기
    # document.documentElement.scrollHeight 반환 => 핸재 페이지의 전체 높이
    last_page_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        #window.scrollTo(x, y) : 윈도우 페이지를 특정 좌표까지 스크롤 / x는 가로축, y는 세로축
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(random.uniform(2, 5))
        new_page_height = driver.execute_script("return document.documentElement.scrollHeight") #스크롤 한 후 페이지의 높이

        if new_page_height == last_page_height: #스크롤 전 페이지높이와 스크롤 후 페이지 높이가 같을 경우 로딩을 한 번 더 기다린 후 그래도 같다면 break
            time.sleep(random.uniform(5, 10))
            if new_page_height == driver.execute_script("return document.documentElement.scrollHeight"):
                break
        else:
            last_page_height = new_page_height

In [ ]:
infinite_loop()

### 코드 진행시 가격 누락 발생
- 코드 진행 시 일부 가격 누락이 발생하여 다른 코드로 크롤링

In [ ]:
'''
item_selector = "a > div > div.m-product-listing__meta > div "
price_selector = "a > div > div.m-product-listing__meta > p > span > strong"
links_selector = "#content > div:nth-child(3) > div > div.o-search__wrapper > ul > li > div > a"
img_selector = 'div > div.m-product-listing__img.m-product-listing__img--quick-view > div:nth-child(1) > img' 

actions = driver.find_element(By.CSS_SELECTOR, 'body')

product_name = []
price = []
href = []
img = []

def CELINE_crawling():

    #상품명
    for i in driver.find_elements(By.CSS_SELECTOR, item_selector):
        print("Product_name :" , i.text)
        product_name.append(i.text)

    #상품 가격
    for i in driver.find_elements(By.CSS_SELECTOR, price_selector): 
        print("Price :" , i.text)
        price.append(i.text)

    #링크
    #모든 a 태그 가져오기
    
    `# href 속성 값 가져오기
    for link_element in driver.find_elements(By.CSS_SELECTOR, links_selector):
        href_value = link_element.get_attribute('href')
        print('href_value :' , href_value)
        href.append(href_value)

    #이미지 url

    # srcset 속성 값 가져오기
    # 마우스 hover이 적용되지 않은 가장 해상도가 좋은 이미지 주소 가져오기
    for img_element in driver.find_elements(By.CSS_SELECTOR, img_selector):
        srcset_value = img_element.get_attribute("srcset")
        print("img_value : " ,srcset_value.split(",")[-1])
        img.append(srcset_value.split(",")[-1])

    time.sleep(random.uniform(0, 3))`

CELINE_crawling()

#driver.quit()

'''

### 제품명, 가격 전부 깔끔하게 나오는 크롤링 코드

In [ ]:
#이름, 색상, 가격 추출 함수
def clean_words(input_str):
    punctuation = '.,;:"“!?_-()[]{}<>@#$%^&*~' #구두점 집합
    #punctuation = string.punctuation #구두점 집합
    word_list = input_str.lower().split('\n')
    word_list = [word.strip(punctuation) for word in word_list]
    if len(word_list) != 3:
        word_list = word_list[:3]
    return word_list

In [ ]:
item_selector = "a > div > div.m-product-listing__meta > div "
links_selector = "#content > div:nth-child(3) > div > div.o-search__wrapper > ul > li > div > a"
img_selector = 'div > div.m-product-listing__img.m-product-listing__img--quick-view > div:nth-child(1) > img' 
list_selector = '#content > div:nth-child(3) > div > div.o-search__wrapper > ul > li'
actions = driver.find_element(By.CSS_SELECTOR, 'body')

product_name_list = []
price_list = []
color_list= []
href_list = []
img_list = []
list_value = []

def CELINE_crawling():

    for i in driver.find_elements(By.CSS_SELECTOR, list_selector):
        print(i.text)
        info = clean_words(i.text)

        product_name,color,price = info
        product_name_list.append(product_name) 
        color_list.append(color)
        price_list.append(price)
        time.sleep(random.uniform(0, 3))
    
    # href 속성 값 가져오기
    for link_element in driver.find_elements(By.CSS_SELECTOR, links_selector):
        href_value = link_element.get_attribute('href')
        print('href_value :' , href_value)
        href_list.append(href_value)
        time.sleep(random.uniform(0, 3))

    #이미지 url

    # srcset 속성 값 가져오기
    # 마우스 hover이 적용되지 않은 가장 해상도가 좋은 이미지 주소 가져오기
    for img_element in driver.find_elements(By.CSS_SELECTOR, img_selector):
        srcset_value = img_element.get_attribute("srcset")
        print("img_value : " ,srcset_value.split(",")[-1])
        img_list.append(srcset_value.split(",")[-1])
        time.sleep(random.uniform(0, 3))
    

CELINE_crawling()

driver.quit()


In [ ]:
#모든 리스트가 똑같이 크롤링 됐는지 확인
Value_List = [product_name_list,price_list,color_list,href_list,img_list]
for i in Value_List:
    print(len(i))
        

주소 문자열만 추출

In [ ]:
for img in img_list:
    print(img[:-6])

## 데이터 프레임 생성

In [ ]:
data = {
    "제품명": product_name_list,
    "가격": price_list,
    "색깔": color_list,
    "상세주소": href_list,
    "이미지 주소": img_list
}

df = pd.DataFrame(data)

# 데이터프레임 출력
print(df)

# 저장

In [56]:
df.to_csv("CELINE.csv", index=False)

### 이후 주소를 통해 이미지 로딩 및 인코딩은 코랩 환경에서 진행